# Get KG entity embeddings

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer  = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")
bert_model = AutoModel.from_pretrained("FacebookAI/roberta-large")
device = torch.device('cuda')
bert_model.to(device)
bert_model.eval()

repo_root = '../../../'

/home/jl254/miniconda3/envs/gat2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jl254/miniconda3/envs/gat2/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
with open(f"{repo_root}/data/dbpedia/concept.txt") as f:
    names = [line.strip() for line in f]

In [5]:
from tqdm import tqdm
import numpy as np

embs = []
tensors = tokenizer(names, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    for i, j in enumerate(tqdm(names)):
        outputs = bert_model(input_ids=tensors["input_ids"][i:i+1].to(device), 
                               attention_mask=tensors['attention_mask'][i:i+1].to(device))
        out = np.array(outputs[1].squeeze().tolist()).reshape((1, -1))
        embs.append(out)
embs = np.concatenate(embs)
np.save(f"{repo_root}/data/dbpedia/ent_emb.npy", embs)

100%|██████████| 3634/3634 [00:54<00:00, 66.44it/s]
